In [1]:
# Module
import pandas as pd
import json
import urllib
import csv
import xml.etree.ElementTree as ET
import sys
import requests
import pprint
from os import name
import bs4
from lxml import html
from urllib.parse import urlencode, quote_plus, unquote
import re
import zipfile
import io
import os
import time
import calendar
import shutil

In [2]:

openkrx_key="4AA045BD69864FB588050FBE3C820B4CF3B12897"
url="http://data-dbg.krx.co.kr/svc/apis/sto/stk_bydd_trd?"

In [52]:
#2015년부터 2022년까지 Data모으기
fail_days=[]
day="";
for year in range(2023,2024):
    for month in range(1,4):
        for day in range(1,32):
            day=str(year)+str(month).zfill(2)+str(day).zfill(2)
            params={'basDd':day,'AUTH_KEY':openkrx_key}
            time.sleep(0.1)
            response=requests.get(url,params=params)
            temp_json=response.json()
            file_path="./krx_stk_trd/"+str(year)+"/"+str(month).zfill(2)+"/"+day+"_stk_trd"
            if(temp_json['OutBlock_1']!=[]):
                with open(file_path,'w') as jsonfile:
                    json.dump(temp_json,jsonfile)
            else:
                fail_days.append(day)
                

In [46]:
# for year in range(2016,2024):
#     folder_path="./krx_stk_trd/"+str(year)
#     try: 
#         if not os.path.exists(folder_path):
#             os.makedirs(folder_path)
#     except OSError:
#         print("Error: Creating driectory")

In [11]:
# for year in range(2015,2016):
#     for month in range(1,13):
#         folder_path="./krx_stk_trd/"+str(year)+"/"+str(month).zfill(2)
#         try: 
#             if not os.path.exists(folder_path):
#                 os.makedirs(folder_path)
#         except OSError:
#             print("Error: Creating driectory")

In [7]:
# OutBlock_1	Block	repeat: multi
# BAS_DD	string()	기준일자
# ISU_CD	string()	종목코드
# ISU_NM	string()	종목명
# MKT_NM	string()	시장구분
# SECT_TP_NM	string()	소속부
# TDD_CLSPRC	string()	종가
# CMPPREVDD_PRC	string()	대비
# FLUC_RT	string()	등락률
# TDD_OPNPRC	string()	시가
# TDD_HGPRC	string()	고가
# TDD_LWPRC	string()	저가
# ACC_TRDVOL	string()	거래량
# ACC_TRDVAL	string()	거래대금
# MKTCAP	string()	시가총액
# LIST_SHRS	string()	상장주식수
file_path='./krx_stk_trd/2023/03/20230314_stk_trd'
with open(file_path,'r') as file:
    data=json.load(file)
    print(data)

{'OutBlock_1': [{'BAS_DD': '20230314', 'ISU_CD': '095570', 'ISU_NM': 'AJ네트웍스', 'MKT_NM': 'KOSPI', 'SECT_TP_NM': '-', 'TDD_CLSPRC': '5010', 'CMPPREVDD_PRC': '-180', 'FLUC_RT': '-3.47', 'TDD_OPNPRC': '5190', 'TDD_HGPRC': '5190', 'TDD_LWPRC': '4970', 'ACC_TRDVOL': '288341', 'ACC_TRDVAL': '1448571670', 'MKTCAP': '234579697950', 'LIST_SHRS': '46822295'}, {'BAS_DD': '20230314', 'ISU_CD': '006840', 'ISU_NM': 'AK홀딩스', 'MKT_NM': 'KOSPI', 'SECT_TP_NM': '-', 'TDD_CLSPRC': '15450', 'CMPPREVDD_PRC': '-790', 'FLUC_RT': '-4.86', 'TDD_OPNPRC': '16240', 'TDD_HGPRC': '16240', 'TDD_LWPRC': '15450', 'ACC_TRDVOL': '12848', 'ACC_TRDVAL': '201331490', 'MKTCAP': '204674817450', 'LIST_SHRS': '13247561'}, {'BAS_DD': '20230314', 'ISU_CD': '027410', 'ISU_NM': 'BGF', 'MKT_NM': 'KOSPI', 'SECT_TP_NM': '-', 'TDD_CLSPRC': '4175', 'CMPPREVDD_PRC': '-195', 'FLUC_RT': '-4.46', 'TDD_OPNPRC': '4360', 'TDD_HGPRC': '4365', 'TDD_LWPRC': '4160', 'ACC_TRDVOL': '149065', 'ACC_TRDVAL': '629465020', 'MKTCAP': '399617602425', 'LIST

In [8]:
code_list=[]
wu_list=[]
for element in data['OutBlock_1']:
    code_list.append(element['ISU_CD'])

print(len(code_list))


943


In [9]:
#krx_stk_trd의 모든 file 종목코드 별로 정리하기
#df_list에 df 899개 만들기
stk_trd_df_list={}
stk_trd_df=pd.DataFrame(columns=('BAS_DD','ISU_CD','ISU_NM','MKT_NM','SECT_TP_NM','TDD_CLSPRC','CMPPREVDD_PRC','FLUC_RT','TDD_OPNPRC','TDD_LWPRC','ACC_TRDVOL','ACC_TRDVAL','MKTCAP','LIST_SHRS'))
for element in data['OutBlock_1']:
    stk_trd_df_list[element['ISU_CD']]=stk_trd_df
    row={'BAS_DD':element['BAS_DD'],
                             'ISU_CD':element['ISU_CD'],
                             'ISU_NM':element['ISU_NM'],
                             'MKT_NM':element['MKT_NM'],
                             'SECT_TP_NM':element['SECT_TP_NM'],
                             'TDD_CLSPRC':element['TDD_CLSPRC'],
                             'CMPPREVDD_PRC':element['CMPPREVDD_PRC'],
                             'FLUC_RT':element['FLUC_RT'],
                             'TDD_OPNPRC':element['TDD_OPNPRC'],
                             'TDD_LWPRC':element['TDD_LWPRC'],
                             'ACC_TRDVOL':element['ACC_TRDVOL'],
                             'ACC_TRDVAL':element['ACC_TRDVAL'],
                             'MKTCAP':element['MKTCAP'],
                             'LIST_SHRS':element['LIST_SHRS']}
    
    row=pd.DataFrame(row,index=[0])
    print(row)
#print(stk_trd_df_list)
print(stk_trd_df_list['068870'])

     BAS_DD  ISU_CD  ISU_NM MKT_NM SECT_TP_NM TDD_CLSPRC CMPPREVDD_PRC  \
0  20230314  095570  AJ네트웍스  KOSPI          -       5010          -180   

  FLUC_RT TDD_OPNPRC TDD_LWPRC ACC_TRDVOL  ACC_TRDVAL        MKTCAP LIST_SHRS  
0   -3.47       5190      4970     288341  1448571670  234579697950  46822295  
     BAS_DD  ISU_CD ISU_NM MKT_NM SECT_TP_NM TDD_CLSPRC CMPPREVDD_PRC FLUC_RT  \
0  20230314  006840  AK홀딩스  KOSPI          -      15450          -790   -4.86   

  TDD_OPNPRC TDD_LWPRC ACC_TRDVOL ACC_TRDVAL        MKTCAP LIST_SHRS  
0      16240     15450      12848  201331490  204674817450  13247561  
     BAS_DD  ISU_CD ISU_NM MKT_NM SECT_TP_NM TDD_CLSPRC CMPPREVDD_PRC FLUC_RT  \
0  20230314  027410    BGF  KOSPI          -       4175          -195   -4.46   

  TDD_OPNPRC TDD_LWPRC ACC_TRDVOL ACC_TRDVAL        MKTCAP LIST_SHRS  
0       4360      4160     149065  629465020  399617602425  95716791  
     BAS_DD  ISU_CD  ISU_NM MKT_NM SECT_TP_NM TDD_CLSPRC CMPPREVDD_PRC  \
0  2023

     BAS_DD  ISU_CD ISU_NM MKT_NM SECT_TP_NM TDD_CLSPRC CMPPREVDD_PRC FLUC_RT  \
0  20230314  092220    KEC  KOSPI          -       2340          -190   -7.51   

  TDD_OPNPRC TDD_LWPRC ACC_TRDVOL  ACC_TRDVAL        MKTCAP  LIST_SHRS  
0       2505      2335    3008412  7212306010  338745749940  144763141  
     BAS_DD  ISU_CD ISU_NM MKT_NM SECT_TP_NM TDD_CLSPRC CMPPREVDD_PRC FLUC_RT  \
0  20230314  016380   KG스틸  KOSPI          -       8490          -330   -3.74   

  TDD_OPNPRC TDD_LWPRC ACC_TRDVOL  ACC_TRDVAL        MKTCAP  LIST_SHRS  
0       8690      8490     850068  7329264530  849075535530  100008897  
     BAS_DD  ISU_CD ISU_NM MKT_NM SECT_TP_NM TDD_CLSPRC CMPPREVDD_PRC FLUC_RT  \
0  20230314  001390  KG케미칼  KOSPI          -      29300           650    2.27   

  TDD_OPNPRC TDD_LWPRC ACC_TRDVOL   ACC_TRDVAL        MKTCAP LIST_SHRS  
0      28400     28200     806938  23672262800  406276612800  13866096  
     BAS_DD  ISU_CD  ISU_NM MKT_NM SECT_TP_NM TDD_CLSPRC CMPPREVDD_PRC  \

     BAS_DD  ISU_CD ISU_NM MKT_NM SECT_TP_NM TDD_CLSPRC CMPPREVDD_PRC FLUC_RT  \
0  20230314  001510   SK증권  KOSPI          -        693           -30   -4.15   

  TDD_OPNPRC TDD_LWPRC ACC_TRDVOL  ACC_TRDVAL        MKTCAP  LIST_SHRS  
0        718       693    4527036  3174483208  327504988503  472590171  
     BAS_DD  ISU_CD ISU_NM MKT_NM SECT_TP_NM TDD_CLSPRC CMPPREVDD_PRC FLUC_RT  \
0  20230314  001515  SK증권우  KOSPI          -       2640           -70   -2.58   

  TDD_OPNPRC TDD_LWPRC ACC_TRDVOL ACC_TRDVAL       MKTCAP LIST_SHRS  
0       2705      2630       7878   20902765  10329036960   3912514  
     BAS_DD  ISU_CD ISU_NM MKT_NM SECT_TP_NM TDD_CLSPRC CMPPREVDD_PRC FLUC_RT  \
0  20230314  285130  SK케미칼  KOSPI          -      74800         -2600   -3.36   

  TDD_OPNPRC TDD_LWPRC ACC_TRDVOL  ACC_TRDVAL         MKTCAP LIST_SHRS  
0      76400     74700      80341  6058637800  1290582968400  17253783  
     BAS_DD  ISU_CD  ISU_NM MKT_NM SECT_TP_NM TDD_CLSPRC CMPPREVDD_PRC  \
0  20

     BAS_DD  ISU_CD ISU_NM MKT_NM SECT_TP_NM TDD_CLSPRC CMPPREVDD_PRC FLUC_RT  \
0  20230314  000320  노루홀딩스  KOSPI          -       9530          -310   -3.15   

  TDD_OPNPRC TDD_LWPRC ACC_TRDVOL ACC_TRDVAL        MKTCAP LIST_SHRS  
0       9810      9530      25529  245609370  126664669030  13291151  
     BAS_DD  ISU_CD  ISU_NM MKT_NM SECT_TP_NM TDD_CLSPRC CMPPREVDD_PRC  \
0  20230314  000325  노루홀딩스우  KOSPI          -      21800          -650   

  FLUC_RT TDD_OPNPRC TDD_LWPRC ACC_TRDVOL ACC_TRDVAL      MKTCAP LIST_SHRS  
0   -2.90      22300     21600        820   17880000  5309586200    243559  
     BAS_DD  ISU_CD ISU_NM MKT_NM SECT_TP_NM TDD_CLSPRC CMPPREVDD_PRC FLUC_RT  \
0  20230314  006280    녹십자  KOSPI          -     116700         -2700   -2.26   

  TDD_OPNPRC TDD_LWPRC ACC_TRDVOL  ACC_TRDVAL         MKTCAP LIST_SHRS  
0     119600    116500      29274  3429112500  1363818984600  11686538  
     BAS_DD  ISU_CD  ISU_NM MKT_NM SECT_TP_NM TDD_CLSPRC CMPPREVDD_PRC  \
0  202303

     BAS_DD  ISU_CD    ISU_NM MKT_NM SECT_TP_NM TDD_CLSPRC CMPPREVDD_PRC  \
0  20230314  000640  동아쏘시오홀딩스  KOSPI          -      92000          -700   

  FLUC_RT TDD_OPNPRC TDD_LWPRC ACC_TRDVOL ACC_TRDVAL        MKTCAP LIST_SHRS  
0   -0.76      93000     91100       3717  340714300  584099996000   6348913  
     BAS_DD  ISU_CD ISU_NM MKT_NM SECT_TP_NM TDD_CLSPRC CMPPREVDD_PRC FLUC_RT  \
0  20230314  170900  동아에스티  KOSPI          -      57300         -2500   -4.18   

  TDD_OPNPRC TDD_LWPRC ACC_TRDVOL ACC_TRDVAL        MKTCAP LIST_SHRS  
0      59700     56900       7253  421431400  483989492400   8446588  
     BAS_DD  ISU_CD ISU_NM MKT_NM SECT_TP_NM TDD_CLSPRC CMPPREVDD_PRC FLUC_RT  \
0  20230314  028100   동아지질  KOSPI          -      11630          -420   -3.49   

  TDD_OPNPRC TDD_LWPRC ACC_TRDVOL ACC_TRDVAL        MKTCAP LIST_SHRS  
0      11770     11530      42876  501136590  163432272980  14052646  
     BAS_DD  ISU_CD ISU_NM MKT_NM SECT_TP_NM TDD_CLSPRC CMPPREVDD_PRC FLUC_RT  

...

     BAS_DD  ISU_CD ISU_NM MKT_NM SECT_TP_NM TDD_CLSPRC CMPPREVDD_PRC FLUC_RT  \
0  20230314  010640   진양폴리  KOSPI          -       3880           -35   -0.89   

  TDD_OPNPRC TDD_LWPRC ACC_TRDVOL ACC_TRDVAL       MKTCAP LIST_SHRS  
0       3895      3810      34699  133486965  38800000000  10000000  
     BAS_DD  ISU_CD ISU_NM MKT_NM SECT_TP_NM TDD_CLSPRC CMPPREVDD_PRC FLUC_RT  \
0  20230314  100250  진양홀딩스  KOSPI          -       3030           -50   -1.62   

  TDD_OPNPRC TDD_LWPRC ACC_TRDVOL ACC_TRDVAL        MKTCAP LIST_SHRS  
0       3075      3005      57058  172815025  169362734760  55895292  
     BAS_DD  ISU_CD ISU_NM MKT_NM SECT_TP_NM TDD_CLSPRC CMPPREVDD_PRC FLUC_RT  \
0  20230314  051630   진양화학  KOSPI          -       2800          -165   -5.56   

  TDD_OPNPRC TDD_LWPRC ACC_TRDVOL ACC_TRDVAL       MKTCAP LIST_SHRS  
0       2940      2800      46590  133268710  44240000000  15800000  
     BAS_DD  ISU_CD ISU_NM MKT_NM SECT_TP_NM TDD_CLSPRC CMPPREVDD_PRC FLUC_RT  \
0  20230

     BAS_DD  ISU_CD ISU_NM MKT_NM SECT_TP_NM TDD_CLSPRC CMPPREVDD_PRC FLUC_RT  \
0  20230314  010820    퍼스텍  KOSPI          -       3315           230    7.46   

  TDD_OPNPRC TDD_LWPRC ACC_TRDVOL   ACC_TRDVAL        MKTCAP LIST_SHRS  
0       3325      3230   26734783  92802317010  159323739900  48061460  
     BAS_DD  ISU_CD ISU_NM MKT_NM SECT_TP_NM TDD_CLSPRC CMPPREVDD_PRC FLUC_RT  \
0  20230314  016800    퍼시스  KOSPI          -      29400          -400   -1.34   

  TDD_OPNPRC TDD_LWPRC ACC_TRDVOL ACC_TRDVAL        MKTCAP LIST_SHRS  
0      29800     28300       2577   75531050  338100000000  11500000  
     BAS_DD  ISU_CD  ISU_NM MKT_NM SECT_TP_NM TDD_CLSPRC CMPPREVDD_PRC  \
0  20230314  001020  페이퍼코리아  KOSPI          -       1162           -79   

  FLUC_RT TDD_OPNPRC TDD_LWPRC ACC_TRDVOL ACC_TRDVAL       MKTCAP LIST_SHRS  
0   -6.37       1220      1162     168231  202287458  55272633220  47566810  
     BAS_DD  ISU_CD ISU_NM MKT_NM SECT_TP_NM TDD_CLSPRC CMPPREVDD_PRC FLUC_RT  \


     BAS_DD  ISU_CD ISU_NM MKT_NM SECT_TP_NM TDD_CLSPRC CMPPREVDD_PRC FLUC_RT  \
0  20230314  003300  한일홀딩스  KOSPI          -      10810          -290   -2.61   

  TDD_OPNPRC TDD_LWPRC ACC_TRDVOL ACC_TRDVAL        MKTCAP LIST_SHRS  
0      11040     10810       6711   72854370  333303476040  30832884  
     BAS_DD  ISU_CD ISU_NM MKT_NM SECT_TP_NM TDD_CLSPRC CMPPREVDD_PRC FLUC_RT  \
0  20230314  051600  한전KPS  KOSPI          -      35500          -950   -2.61   

  TDD_OPNPRC TDD_LWPRC ACC_TRDVOL  ACC_TRDVAL         MKTCAP LIST_SHRS  
0      36550     35200     203869  7310379000  1597500000000  45000000  
     BAS_DD  ISU_CD ISU_NM MKT_NM SECT_TP_NM TDD_CLSPRC CMPPREVDD_PRC FLUC_RT  \
0  20230314  052690   한전기술  KOSPI          -      67100         -3600   -5.09   

  TDD_OPNPRC TDD_LWPRC ACC_TRDVOL   ACC_TRDVAL         MKTCAP LIST_SHRS  
0      70100     66800     327389  22449449000  2564562000000  38220000  
     BAS_DD  ISU_CD ISU_NM MKT_NM SECT_TP_NM TDD_CLSPRC CMPPREVDD_PRC FLUC_

     BAS_DD  ISU_CD  ISU_NM MKT_NM SECT_TP_NM TDD_CLSPRC CMPPREVDD_PRC  \
0  20230314  298050  효성첨단소재  KOSPI          -     406000        -14500   

  FLUC_RT TDD_OPNPRC TDD_LWPRC ACC_TRDVOL  ACC_TRDVAL         MKTCAP LIST_SHRS  
0   -3.45     421500    406000      23110  9560761000  1818858888000   4479948  
     BAS_DD  ISU_CD ISU_NM MKT_NM SECT_TP_NM TDD_CLSPRC CMPPREVDD_PRC FLUC_RT  \
0  20230314  298020  효성티앤씨  KOSPI          -     412500         -7500   -1.79   

  TDD_OPNPRC TDD_LWPRC ACC_TRDVOL  ACC_TRDVAL         MKTCAP LIST_SHRS  
0     418000    405500      22970  9471904500  1785168825000   4327682  
     BAS_DD  ISU_CD ISU_NM MKT_NM SECT_TP_NM TDD_CLSPRC CMPPREVDD_PRC FLUC_RT  \
0  20230314  298000   효성화학  KOSPI          -     126000         -7000   -5.26   

  TDD_OPNPRC TDD_LWPRC ACC_TRDVOL  ACC_TRDVAL        MKTCAP LIST_SHRS  
0     133300    126000      33098  4255413800  401955876000   3190126  
     BAS_DD  ISU_CD ISU_NM MKT_NM SECT_TP_NM TDD_CLSPRC CMPPREVDD_PRC FLU

KeyError: '068870'

In [10]:
#krx_stk_trd의 모든 file 종목코드 별로 정리하기 
#df_list에 하나하나씩 담기
for year in range(2015,2024):
    for month in range(1,13):
        list_path='./krx_stk_trd/'+str(year)+'/'+str(month).zfill(2)
        listdir=os.listdir(list_path)
        for file_name in listdir:
            #print(file_name)
            if(file_name==".ipynb_checkpoints"):
                pass
            else:
                file_path=list_path+'/'+file_name
                with open(file_path,'r') as file:
                    data=json.load(file)

                for element in data['OutBlock_1']:
                    #print(element)
                    code=element['ISU_CD']
                    row={'BAS_DD':element['BAS_DD'],
                         'ISU_CD':element['ISU_CD'],
                         'ISU_NM':element['ISU_NM'],
                         'MKT_NM':element['MKT_NM'],
                         'SECT_TP_NM':element['SECT_TP_NM'],
                         'TDD_CLSPRC':element['TDD_CLSPRC'],
                         'CMPPREVDD_PRC':element['CMPPREVDD_PRC'],
                         'FLUC_RT':element['FLUC_RT'],
                         'TDD_OPNPRC':element['TDD_OPNPRC'],
                         'TDD_LWPRC':element['TDD_LWPRC'],
                         'ACC_TRDVOL':element['ACC_TRDVOL'],
                         'ACC_TRDVAL':element['ACC_TRDVAL'],
                         'MKTCAP':element['MKTCAP'],
                         'LIST_SHRS':element['LIST_SHRS']}
                    row=pd.DataFrame(row,index=[0])
                    if code in stk_trd_df_list:
                        stk_trd_df_list[code]=pd.concat([stk_trd_df_list[code],row],ignore_index=True)

                file.close()
       

In [12]:
print(stk_trd_df_list)

{'095570':         BAS_DD  ISU_CD  ISU_NM MKT_NM SECT_TP_NM TDD_CLSPRC CMPPREVDD_PRC  \
0     20150821  095570  AJ네트웍스  KOSPI          -      35500          4600   
1     20150824  095570  AJ네트웍스  KOSPI          -      32400         -3100   
2     20150825  095570  AJ네트웍스  KOSPI          -      31100         -1300   
3     20150826  095570  AJ네트웍스  KOSPI          -      35450          4350   
4     20150827  095570  AJ네트웍스  KOSPI          -      37850          2400   
...        ...     ...     ...    ...        ...        ...           ...   
1855  20230308  095570  AJ네트웍스  KOSPI          -       5400            50   
1856  20230309  095570  AJ네트웍스  KOSPI          -       5380           -20   
1857  20230310  095570  AJ네트웍스  KOSPI          -       5300           -80   
1858  20230313  095570  AJ네트웍스  KOSPI          -       5190          -110   
1859  20230314  095570  AJ네트웍스  KOSPI          -       5010          -180   

     FLUC_RT TDD_OPNPRC TDD_LWPRC ACC_TRDVOL   ACC_TRDVAL       

In [11]:
temp_dir_list={"1":1,"2":2,"3":3}
for element in temp_dir_list:
    print(element)

1
2
3


In [13]:
for corp_code in stk_trd_df_list:
    stk_trd_df_list[corp_code].to_csv("./corpcode_stk_trd/"+corp_code+".csv",mode='w')

In [16]:
print(len(os.listdir("./corpcode_stk_trd")))

943
